In [24]:
#pip install plotly


In [1]:
#Importando as bibliotecas
import datetime

import numpy                as np
import pandas               as pd
 
import plotly.express       as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importando base de dados
#Carregando o parquet
base_dados = pd.read_parquet('base_consolidada.parquet')

#hora atual
print(datetime.datetime.now())

2023-06-27 10:08:34.427343


In [4]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4135523 entries, 0 to 4135522
Data columns (total 25 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   Número do Auto                 object
 1   Data da Infração (DD/MM/AAAA)  object
 2   Indicador de Abordagem         object
 3   Assinatura do Auto             object
 4   Sentido Trafego                object
 5   UF Infração                    object
 6   BR Infração                    object
 7   Km Infração                    object
 8   Município                      object
 9   Indicador Veiculo Estrangeiro  object
 10  UF Placa                       object
 11  Descrição Especie Veículo      object
 12  Descrição Marca Veículo        object
 13  Descrição Tipo Veículo         object
 14  Descrição Modelo Veiculo       object
 15  Código da Infração             object
 16  Descrição Abreviada Infração   object
 17  Enquadramento da Infração      object
 18  Início Vigência da Inf

# Tratando e convertendo os dados

In [5]:
# Nulos
base_dados.isnull().sum()

Número do Auto                   0
Data da Infração (DD/MM/AAAA)    0
Indicador de Abordagem           0
Assinatura do Auto               0
Sentido Trafego                  0
UF Infração                      0
BR Infração                      0
Km Infração                      0
Município                        0
Indicador Veiculo Estrangeiro    0
UF Placa                         0
Descrição Especie Veículo        0
Descrição Marca Veículo          0
Descrição Tipo Veículo           0
Descrição Modelo Veiculo         0
Código da Infração               0
Descrição Abreviada Infração     0
Enquadramento da Infração        0
Início Vigência da Infração      0
Fim Vigência Infração            0
Medição Infração                 0
Hora Infração                    0
Medição Considerada              0
Excesso Verificado               0
Qtd Infrações                    0
dtype: int64

In [6]:
#valores unicos
base_dados.nunique()

Número do Auto                   4135523
Data da Infração (DD/MM/AAAA)        365
Indicador de Abordagem                 2
Assinatura do Auto                     2
Sentido Trafego                        2
UF Infração                           27
BR Infração                          119
Km Infração                         1223
Município                           2078
Indicador Veiculo Estrangeiro          3
UF Placa                              90
Descrição Especie Veículo             14
Descrição Marca Veículo            13173
Descrição Tipo Veículo                24
Descrição Modelo Veiculo           25719
Código da Infração                   378
Descrição Abreviada Infração         378
Enquadramento da Infração            239
Início Vigência da Infração            8
Fim Vigência Infração                  1
Medição Infração                       5
Hora Infração                         24
Medição Considerada                11978
Excesso Verificado                  8518
Qtd Infrações   

In [7]:
base_dados.head(3)

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,Indicador Veiculo Estrangeiro,...,Código da Infração,Descrição Abreviada Infração,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações
0,0b23298312cfcd6,2022-01-01,S,N,C,BA,116,818,VITORIA DA CONQUISTA,N,...,74630,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,13,81,21,1
1,53094c3e97c04ff,2022-01-01,S,N,C,BA,116,818,VITORIA DA CONQUISTA,N,...,74630,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,15,72,12,1
2,e7ba581a289d3c9,2022-01-01,S,N,D,RJ,101,325,NITEROI,N,...,74550,Transitar em velocidade superior à máxima perm...,218 I,2016-11-01,nan,Veiculo,15,92,12,1


### Total de multas


In [8]:
total_multa = int(base_dados.shape[0])
print(f'Total de multas nos ultimos 12 meses: {total_multa}')

Total de multas nos ultimos 12 meses: 4135523


## Convertendo e modelando os Dados


In [9]:
#Datas
print(base_dados['Data da Infração (DD/MM/AAAA)'].dtype)

base_dados['Data da Infração (DD/MM/AAAA)'] = pd.to_datetime(base_dados['Data da Infração (DD/MM/AAAA)'])


print(base_dados['Data da Infração (DD/MM/AAAA)'].dtype)


object
datetime64[ns]


In [10]:
max_dia = base_dados['Data da Infração (DD/MM/AAAA)'].max()
min_dia = base_dados['Data da Infração (DD/MM/AAAA)'].min()

print(f'max data: {max_dia}, Min data: {min_dia}')

max data: 2022-12-31 00:00:00, Min data: 2022-01-01 00:00:00


In [11]:
#Filtrando os dias e multas
multa_dia = base_dados['Data da Infração (DD/MM/AAAA)'].value_counts().sort_index().reset_index()

multa_dia['mes'] = pd.to_datetime(multa_dia['Data da Infração (DD/MM/AAAA)']).dt.month
multa_dia['dia'] = pd.to_datetime(multa_dia['Data da Infração (DD/MM/AAAA)']).dt.day

#Renomeando as colunas
multa_dia.columns = ['Data', 'Tot_Multas', 'Mes', 'Dia']

#Média movel de 7 dias
multa_dia['Media_Movel'] = multa_dia['Tot_Multas'].rolling(7).mean()

multa_dia


,Data,Tot_Multas,Mes,Dia,Media_Movel
0,2022-01-01,8533,1,1,NaN
1,2022-01-02,13146,1,2,NaN
2,2022-01-03,12012,1,3,NaN
3,2022-01-04,11248,1,4,NaN
4,2022-01-05,11159,1,5,NaN
...,...,...,...,...,...
360,2022-12-27,12077,12,27,13297.142857
361,2022-12-28,11332,12,28,13104.714286
362,2022-12-29,12226,12,29,13149.571429
363,2022-12-30,15801,12,30,13046.428571


In [12]:
multa_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Data         365 non-null    datetime64[ns]
 1   Tot_Multas   365 non-null    int64         
 2   Mes          365 non-null    int32         
 3   Dia          365 non-null    int32         
 4   Media_Movel  359 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int32(2), int64(1)
memory usage: 11.5 KB


## Visualizando os dados

In [12]:
px.line(
    multa_dia,
    x='Data',
    y='Tot_Multas'
)

In [13]:
# Go
go.Figure(
    go.Scatter(
        x=multa_dia['Data'],
        y=multa_dia['Tot_Multas']
    )
)

In [14]:
#Grid
Grid = make_subplots( rows=1, cols=2)

Grid.add_trace(
    go.Scatter(
        x=multa_dia['Data'],
        y=multa_dia['Tot_Multas'],
        mode='lines',
        name='Quantidade'
        ),
        row=1, col=1
)

Grid.add_trace(
    go.Scatter(
        x=multa_dia['Data'],
        y=multa_dia['Media_Movel'],
        mode='lines',
        name='Média Movel'
        ),
        row=1, col=2
)

#Ajuste de Layout
Grid.update_layout(
    #Titulo
    title='Sistema de Grid',

    #legenda
    showlegend=True,

    #Ajust plotly
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )

)

Grid

In [15]:
#Grid
Grid = make_subplots( rows=1, cols=1)

Grid.add_trace(
    go.Scatter(
        x=multa_dia['Data'],
        y=multa_dia['Tot_Multas'],
        mode='lines',
        name='Quantidade'
        ),
        row=1, col=1
)

Grid.add_trace(
    go.Scatter(
        x=multa_dia['Data'],
        y=multa_dia['Media_Movel'],
        mode='lines',
        name='Média Movel'
        ),
        row=1, col=1
)

#Ajuste de Layout
Grid.update_layout(
    #Titulo
    title='Analise Multas Diarias',

    #legenda
    showlegend=True,

    #Ajust plotly
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )

)

Grid

In [16]:
#Distribuição 
px.box(
    multa_dia,
    x='Mes',
    y='Tot_Multas',
    color='Mes',
    title='Distribuição de multas por mês'
)

## Visão Micro

In [13]:
# Estados
acumulado_estados = base_dados['UF Infração'].value_counts()
acumulado_estados_perc = base_dados['UF Infração'].value_counts(normalize=True) # percentual

# represe. acumulada, ou seja, o quanto o UF representa no todo
acumulado_estados_acum = base_dados['UF Infração'].value_counts(normalize=True).cumsum() 

# Dicionario com os dados gerados acima
dicionario = {
    'Estados':acumulado_estados.index,
    'Qtd_Multas':acumulado_estados.values,
    'Representação':acumulado_estados_perc.values,
    'Acumulado':acumulado_estados_acum.values
}

#Criação do DF
tabela_estados = pd.DataFrame( dicionario )

tabela_estados.head(3)

,Estados,Qtd_Multas,Representação,Acumulado
0,MG,448672,0.108492,0.108492
1,RJ,418981,0.101313,0.209805
2,BA,417092,0.100856,0.310661


In [18]:
px.funnel(
    tabela_estados[tabela_estados['Acumulado'] < 0.5],
    y='Estados',
    x='Qtd_Multas',
    title='Concentração dos 50%'
)

In [19]:
px.bar(
    tabela_estados,
    x='Estados',
    y='Acumulado',
    title='Multas Acumuladas'
)

In [14]:
# Ca
base_dados['Mês'] = base_dados['Data da Infração (DD/MM/AAAA)'].dt.month
base_dados.head(3)

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,Indicador Veiculo Estrangeiro,...,Descrição Abreviada Infração,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações,Mês
0,0b23298312cfcd6,2022-01-01,S,N,C,BA,116,818,VITORIA DA CONQUISTA,N,...,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,13,81,21,1,1
1,53094c3e97c04ff,2022-01-01,S,N,C,BA,116,818,VITORIA DA CONQUISTA,N,...,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,15,72,12,1,1
2,e7ba581a289d3c9,2022-01-01,S,N,D,RJ,101,325,NITEROI,N,...,Transitar em velocidade superior à máxima perm...,218 I,2016-11-01,nan,Veiculo,15,92,12,1,1


In [15]:
# Analisando o calor da mautlas
anl_estados_mes = base_dados.groupby(by=['Mês', 'UF Infração']).agg(
    Quantidade = ('Município', 'count')
).reset_index()

anl_estados_mes.head()

,Mês,UF Infração,Quantidade
0,1,AC,1935
1,1,AL,4145
2,1,AM,1960
3,1,AP,1065
4,1,BA,29613


In [16]:
#Criando uma tabela dinamica com python
anl_estados_mes = anl_estados_mes.pivot_table(index='Mês', columns='UF Infração', values='Quantidade')


In [17]:
anl_estados_mes

UF Infração,AC,AL,AM,AP,BA,CE,DF,ES,GO,MA,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
Mês,,,,,,,,,,,,,,,,,,,,,
1,1935,4145,1960,1065,29613,11819,3789,9814,12965,6152,...,28441,47616,5390,7160,931,21648,9393,3347,35375,3320
2,1869,4442,1560,1290,37657,13540,4506,8171,21802,5899,...,26903,43526,6567,8365,1289,24964,13186,3875,37702,3374
3,1626,3892,1451,1136,38322,12586,7749,9687,26626,7029,...,25538,45218,7639,9425,1255,23029,14372,3806,34265,3950
4,1178,4175,2049,1670,42969,15011,13708,12860,25979,5965,...,21734,48050,6544,8750,1523,23871,17591,3901,22893,3168
5,1056,2950,1658,1599,28583,11562,9927,14337,22472,4799,...,18577,41165,6086,8190,703,19892,17870,3193,21472,3518
6,1199,2133,1780,1048,33048,7611,7566,15296,18386,4014,...,12865,47713,5028,7654,610,15623,14244,1053,29731,3272
7,1491,1610,1450,876,32852,6897,8809,14789,18359,5540,...,16983,36345,5192,6754,538,14714,12415,2429,31650,2698
8,1109,2560,1520,1177,34006,10619,3562,15743,17892,7213,...,16044,22501,6374,9161,907,18305,14365,2660,31345,2702
9,1082,4449,1801,1626,34947,10264,3117,16495,17573,6990,...,15283,20197,7185,9287,1013,19967,16094,3539,30824,2282


In [18]:
px.imshow( anl_estados_mes, title='Mapa de calor | Multas mensais por estado 2022')

### Scrapping dos dados

In [19]:
base_dados['Enquadramento da Infração'].value_counts(normalize=True).cumsum()*100

Enquadramento da Infração
218 I        26.437889
167          31.968581
203 V        36.954649
230 * V      41.899029
218 II       46.308581
               ...    
230 XVII     99.999903
252 VII      99.999927
213 I        99.999952
214 III      99.999976
247         100.000000
Name: proportion, Length: 239, dtype: float64

In [3]:
#Loop no site para capturar todas as paginas da lista
url = 'https://www.detran.mg.gov.br/infracoes/consultar-tipos-infracoes/index/index/index/index/index/index/index/index/index/index/index/index/index/lista-de-infracoes?artigo=&descricao=&page='

#Tabela vazia
base_consolidada = pd.DataFrame()

for list_completa in range(1, 24):

    #Construindo o link
    link = f'{url}{list_completa}'

    #Lendo os dados da web
    dados_web = pd.read_html(link)[0]

    #consolidando os dados
    base_consolidada = pd.concat([base_consolidada, dados_web])



In [21]:
base_consolidada.shape

(442, 7)

In [4]:
df = base_consolidada.copy()

In [5]:
df['new_valor'] = df['Valor'].str.replace('.', '').str.replace(',', '')
df['new_valor'].fillna(df['Valor'], inplace=True)
df['new_valor'] = pd.to_numeric( df['new_valor'] )
df['new_valor'] = df['new_valor']/100
df.drop(columns=['Valor', 'Infração', 'Artigo', 'Infrator'], inplace=True)


In [6]:
df.columns

Index(['Código', 'Desd.', 'Pts', 'new_valor'], dtype='object')

In [7]:
df.rename(columns={'Código': 'Código da Infração', 'new_valor': 'Valor'}, inplace=True)

In [26]:
df['Código da Infração'].value_counts()

Código da Infração
5819    8
5452    7
6017    6
6556    5
6009    5
       ..
6793    1
6807    1
6815    1
6858    1
7137    1
Name: count, Length: 272, dtype: int64

In [27]:
df.dtypes

Código da Infração      int64
Desd.                   int64
Pts                     int64
Valor                 float64
dtype: object

In [28]:
df.head()

,Código da Infração,Desd.,Pts,Valor
0,5622,4,3,88.38
1,5622,5,3,88.38
2,5630,0,4,130.16
3,5649,1,4,130.16
4,5649,2,4,130.16


In [8]:
# Agrupando os valores pelo tipo de infração
Tab_Preco = df.groupby(by=['Código da Infração']).mean()['Valor'].reset_index()
Tab_Preco

,Código da Infração,Valor
0,5002,0.00
1,5010,880.41
2,5029,880.41
3,5037,586.94
4,5045,293.47
...,...,...
267,7757,293.47
268,7765,293.47
269,7773,293.47
270,7781,293.47


In [9]:
#Garantindo 4 digitos na coluna
base_dados['Código da Infração'] = base_dados['Código da Infração'].apply(lambda Loop: Loop[0:4])

In [10]:
#Convertendo os tipos
base_dados['Código da Infração'] = pd.to_numeric(base_dados['Código da Infração'])

In [11]:
#Fazendo o merge nas bases de dados
abt_table = pd.merge(base_dados, Tab_Preco, on='Código da Infração', how='left')
abt_table

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,Indicador Veiculo Estrangeiro,...,Descrição Abreviada Infração,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações,Valor
0,0b23298312cfcd6,2022-01-01,S,N,C,BA,116,818,VITORIA DA CONQUISTA,N,...,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,13,81,21,1,195.23
1,53094c3e97c04ff,2022-01-01,S,N,C,BA,116,818,VITORIA DA CONQUISTA,N,...,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,15,72,12,1,195.23
2,e7ba581a289d3c9,2022-01-01,S,N,D,RJ,101,325,NITEROI,N,...,Transitar em velocidade superior à máxima perm...,218 I,2016-11-01,nan,Veiculo,15,92,12,1,130.16
3,e630a12b592c2d7,2022-01-01,S,N,C,SP,116,27,SILVEIRAS,N,...,Transitar em velocidade superior à máxima perm...,218 I,2016-11-01,nan,Veiculo,15,87,7,1,130.16
4,78ff6172803c1ed,2022-01-01,S,N,D,RJ,101,323,NITEROI,N,...,Transitar em velocidade superior à máxima perm...,218 I,2016-11-01,nan,Veiculo,15,81,1,1,130.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135518,f146b3365093012,2022-12-31,S,N,D,MS,262,476,ANASTACIO,N,...,Transitar em velocidade superior à máxima perm...,218 I,2016-11-01,nan,Veiculo,9,108,8,1,130.16
4135519,6cf718bdfd6f418,2022-12-31,S,N,C,MG,50,148,UBERABA,N,...,Transitar em velocidade superior à máxima perm...,218 II,2016-11-01,nan,Veiculo,9,142,32,1,195.23
4135520,77e0e61e610b1a1,2022-12-31,C,N,C,SC,116,4,MAFRA,N,...,"Rec sub test,ex clin, peric ou proc q perm cer...",165-A,2016-11-01,nan,Nenhuma,0,0,0,1,2934.70
4135521,8cbb7dfa2aadfa5,2022-12-31,C,N,C,SE,101,104,SAO CRISTOVAO,N,...,Conduzir o veículo com equipamento obrigatório...,230 IX,2016-11-01,nan,Nenhuma,11,0,0,1,195.23


In [12]:
# Breve visualização
tab_soma = abt_table.groupby(by='UF Infração').agg(
    {'Valor': ['count', 'sum']},
)

tab_soma.columns=tab_soma.columns.droplevel()

tab_soma = tab_soma.reset_index()

tab_soma.head()

,UF Infração,count,sum
0,AC,15531,6.335238e+06
1,AL,40597,1.520225e+07
2,AM,20102,8.382845e+06
3,AP,17726,7.627325e+06
4,BA,417092,1.443661e+08


In [34]:
px.scatter(
    tab_soma,
    x='count',
    y='sum',
    color='UF Infração',
    size='count',
    log_x=True,
    size_max=60,
    title='Burble PLOT'
)


In [35]:
fig = px.scatter(
    tab_soma,
    x='count',
    y='sum',
    color='UF Infração',
    title='Scatter PLOT'
)
fig.update_traces(marker=dict(size=12))

In [36]:
anl_valor_mes = abt_table.groupby(by=['Mês'])['Valor'].sum().reset_index()
anl_valor_mes

,Mês,Valor
0,1,1.209428e+08
1,2,1.399027e+08
2,3,1.449664e+08
3,4,1.489744e+08
4,5,1.200325e+08
5,6,1.159805e+08
6,7,1.121263e+08
7,8,1.118220e+08
8,9,1.209734e+08
9,10,1.392511e+08


In [14]:
abt_table.to_parquet('abt_table_pqt.parquet', index=False)